# Node DP algorithms for degree distribution query

### Helper functions

In [ ]:
import util
import numpy as np
from sklearn.isotonic import IsotonicRegression  
from sklearn.linear_model import LinearRegression 

# Day et al. SIGMOD'16 , algo 1
# projection by edge-addition, variant: output degSeq instead of graph, part of Algo 4
# Edge addition algorithm from empty graph till no edges can be added; keep degree bounded by theta
def deg_list_by_edge_addition(G, theta):    
    num_nodes = len(G.nodes())
    nodes = list(G.nodes()) # the node ids are not strictly from 0 to |nodesNum|-1
    inv_nodes_list = {} # map node id to index
    for id in range(num_nodes):
        v = nodes[id]
        inv_nodes_list[v]=id
    
    nodes_random_indices = np.random.permutation(num_nodes)
    deg_list_temp = np.zeros(num_nodes)
    
    for vid in nodes_random_indices: # edge addition
        v = nodes[vid]
        for u in G.neighbors(v):
            uid = inv_nodes_list[u]
            if uid < vid and deg_list_temp[uid]<theta and deg_list_temp[vid]<theta:
                deg_list_temp[uid] = deg_list_temp[uid]+1
                deg_list_temp[vid] = deg_list_temp[vid]+1
                
    deg_list_temp=sorted(deg_list_temp, reverse=False) # small to large degrees
    
    return deg_list_temp


In [ ]:
# Day et al. SIGMOD'16 
# select an optimal theta (a node degree upper bound) for projection
def learn_theta(G, max_deg, epsilon_theta, epsilon_deg, theta_list):
    prob_list = []
    max_theta = max(theta_list)
    sensitivity = 2.0 * max_theta + 2
    deg_list_max_theta = deg_list_by_edge_addition(G, max_theta)
    
    for theta in theta_list:
        nodes_gt_theta = len([deg for deg in deg_list_max_theta if deg > theta])
        score = -2.0 * nodes_gt_theta - np.sqrt(theta) * (theta+1)/epsilon_deg
        prob = np.exp(epsilon_theta * score /(2.0 * sensitivity))
        prob_list.append(prob)
        
    theta = theta_list[util.sample_prob_list(prob_list)]
    return theta


In [3]:
# Day et al. SIGMOD'16 
# Algo 3: extract and calibrate histogram from cumulative histogram
def extract_hist_from_cum(cum_hist): # cum_hist is noisy cumulative histogram
    theta = len(cum_hist)-1
    hist = np.zeros(len(cum_hist))
    hc=list(cum_hist)
    hc.append(cum_hist[theta])
    hc[0] = 0
    i = 1
    if hc[1] <0:
        hc[1] = 0
    
    while i <= theta:
        if hc[i] <= hc[i+1]:
            hist[i] = max(hc[i] - hc[i-1],0)
            i = i+1
        else: # count i is greater than count i+1, indicating too much noise injection
            iold = i
            jlist = list(reversed(range(i,theta+2)))
            for j in jlist:
                if hc[j-1] < hc[i]:
                    j = min(j,theta)
                    # assign the counts uniformly between bin i to j
                    for k in range(i,j+1):
                        hist[k] = max(0,(hc[j]-hc[i-1])/(j-i+1))
                    i= j+1
                    break 
            if i == iold:
                break
    return hist
    

In [4]:
# Day et al. SIGMOD'16 
# Algo 5 (postprocess for tail destribution), part of Algo 4
# The last bin has a high count which contains the number of nodes with degree "at least" theta in G. 
# Estimate the tail distribution of the noisy histogram, assuming the {theta/2}th bin drops linearly                                                                                                                                                                                                      
def post_process_tail(hist, theta):
    budget = hist[theta] 
    start = int(round(theta/2))
    cbar = 2.0/theta * sum(hist[start:theta])
    
    X = np.array([[x] for x in range(start,theta)])
   
    y = hist[start:theta]
    
    reg = LinearRegression().fit(X,y)
    m = reg.coef_
    b = reg.intercept_ 
    
    for k in range(theta, len(hist)):
        if m<0:
            # This step has an issue when theta is small, then b is small, and counts become negative
            hist[k] = max(b + m * k,0) 
        else:
            hist[k] = cbar
        budget = budget - hist[k]
        if budget <0:
            break
    return hist 


In [5]:
# Day et al. SIGMOD'16 
# Part of Algo 2 : select an optimal theta (max node degree) and partition ()

# theta: optimal node degree upper bound; 
# partition: used for grouping histogram bins and reducing noise effect by averaging
def learn_theta_partition(G, max_deg, epsilon_theta, epsilon_deg, theta_list, partition_list):
    prob_list = []
    max_theta = max(theta_list)
    sensitivity = 6.0 * max_theta + 4
    
    his = util.get_deg_his(G,max_deg)
    deg_list_max_theta = deg_list_by_edge_addition(G,max_theta)
    
    for theta in theta_list:
        nodes_degree_gt_theta = len([deg for deg in deg_list_max_theta if deg >theta])
        lproj = 2.0 * nodes_degree_gt_theta
        # compute lhist
        for partition in partition_list:            
            lhist = 0 
            for i in range(len(partition)-1):
                start = partition[i]
                end = partition[i+1]
                count = end-start
                ave = sum(his[start:end])/count
                diff = sum(abs(his[start:end] - ave)) 
                lhist = lhist + diff + (2.0*theta+1)/epsilon_deg
            score = -1.0 * (lhist + lproj)
            prob = np.exp(epsilon_theta * score /(2.0 * sensitivity))
            prob_list.append(prob)
        
    i = util.sample_prob_list(prob_list)
    theta_index = int(i / len(partition_list))
    partition_index = i % len(partition_list)
    theta= theta_list[theta_index]
    partition = partition_list[partition_index]
    return theta,partition

### Day et al. SIGMOD'16: Algo 2 (theta,Omega-histogram)

In [21]:
# Day et al. SIGMOD'16
# Algo 2 (\theta,\Omega-cumulative)
# Main idea: Select optimal {theta} (node degree bound) and {partition} (noise aggregation) using exponential mechanism
#   , then publishes a noisy histogram with Laplace noise

def nodedp_add_edge_deg_his_part_lap(G, max_deg, epsilon, theta_list, r_list):
    epsilon_deg = epsilon
    max_theta = max(theta_list)
    #Create a set of partitions based on r_list, r should be [1,2]
    partition_list = []
    for r in r_list:
        partition = [0]
        # idea: should group fewer bins on low degrees(usually with larger counts) 
        # and more bins on higher degrees(long tail)
        i = 0 
        k = int(round(np.power(r, i))) 
        while k < max_theta:
            if k > partition[-1]:
                partition.append(k)
            i = i+1
            k = int(round(np.power(r, i)))
        partition.append(max_theta) # we also add the maxDeg for the easy aggregation
        partition_list.append(partition)
        
        #Learning theta and partition
        if len(theta_list) >1 or len(partition_list)>1: # many choices 
            epsilon_theta = epsilon * 0.1
            epsilon_deg = epsilon-epsilon_theta
            (theta, partition) = learn_theta_partition(G, max_deg, epsilon_theta, epsilon_deg, theta_list, partition_list)

        #Edge addition algorithm from empty graph till no edges can be added keep degree bounded by theta
        deg_list_temp = deg_list_by_edge_addition(G,theta)

        # histogram + lap noise on grouped bins
        deg_his = util.deg_seq_to_deg_his(deg_list_temp, theta)
        noisy_deg_his = np.zeros(len(deg_his))
        sens = 2 * theta + 1
        for i in range(len(partition)-1):
            start = partition[i]
            end = partition[i+1]
            if end >= (theta+1):
                end = min(end,theta+1)
            # noisy aggregation
            noisy_average = (sum(deg_his[start:end]) + util.add_laplace([0],sens,epsilon_deg)[0])/(end-start)
            for j in range(start,end):
                noisy_deg_his[j] = noisy_average
            if end >= (theta+1):
                break 

        noisy_deg_his = util.extend_his(noisy_deg_his,max_deg) # make historgram of length {max_deg + 1} by padding 0s
        noisy_deg_his = post_process_tail(noisy_deg_his, theta) # estimate tail distribution
    return noisy_deg_his

## Day et al. SIGMOD'16: Algo 4 (theta-cumulative)

In [7]:
# Main idea: project graph by edge addition such that node degrees are capped by learned theta
# , apply Laplace noise on degree cumulative histogram
def nodedp_add_edge_deg_cum_lap(G, max_degree, epsilon, theta_list):
    theta = theta_list[0]
    epsilon_deg = epsilon
    
    # Learning theta 
    if len(theta_list) > 1: # many choices 
        epsilon_theta = epsilon * 0.1
        epsilon_deg = epsilon-epsilon_theta
        theta = learn_theta(G, max_degree, epsilon_theta, epsilon_deg, theta_list)

    # Edge addition algorithm from empty graph till no edges can be added keep degree bounded by theta
    degree_list = deg_list_by_edge_addition(G,theta)
    
    # cumulative historm + lap noise
    deg_his = util.deg_seq_to_deg_his(degree_list, theta)
    deg_cum = util.pdf_to_cdf(deg_his)
    
    sens = theta + 1 
    noisy_cum = util.add_laplace(deg_cum, sens, epsilon_deg)
    
    noisy_deg_his = extract_hist_from_cum(noisy_cum) 
    noisy_deg_his = util.extend_his(noisy_deg_his, max_degree) # extend truncated histogram to length {max degree + 1}
    noisy_deg_his = post_process_tail(noisy_deg_his, theta) # estimate tail distribution
    
    return noisy_deg_his



### Day et al. SIGMOD'16: Algo 4 variant 

In [8]:
#Day et al. SIGMOD'16
#variant of Algo 4 (\theta-constrained)
def nodedp_add_edge_deg_cum_lap_variant(G, max_deg, epsilon, theta_list):
    theta = theta_list[0]
    epsilon_deg = epsilon
    
    # Learning theta 
    if len(theta_list) >1: #many choices 
        epsilon_theta = epsilon * 0.1
        epsilon_deg = epsilon-epsilon_theta
        theta = learn_theta(G, max_deg, epsilon_theta, epsilon_deg, theta_list)

    #Edge addition algorithm from empty graph till no edges can be added keep degree bounded by theta
    degree_list = deg_list_by_edge_addition(G,theta)
    
    #cumulative historm + lap noise
    deg_his = util.deg_seq_to_deg_his(degree_list, theta)
    deg_cum = util.pdf_to_cdf(deg_his)
    sens = theta + 1 
    noisy_cum = util.add_laplace(deg_cum, sens, epsilon_deg)
    
    # smooth out noise injected using isotonic regression
    noisy_cum = util.post_process_cdf(noisy_cum, len(G.nodes())) # this step is addition to algo 4
    noisy_deg_his = util.cdf_to_pdf(noisy_cum)
    noisy_deg_his = util.extend_his(noisy_deg_his,max_deg)
    noisy_deg_his = post_process_tail(noisy_deg_his, theta)
    
    return noisy_deg_his
    

### Raskhodnikova & Smith, Arxiv'15: Flowgraph based mechanism

In [9]:
# helper: construct and solve flowgraph,
# to obtain a degree sequence with max degree is bounded by theta

import cvxpy as cp
from qpsolvers import solve_qp

def flowgraph(G, theta):
    #https://pypi.org/project/qpsolvers/
    #Raskhodnikova & Smith, Arxiv'15
    #Note: this algo is slow
    num_nodes = len(G.nodes()) 
    num_edges = len(G.edges())

    nn = num_nodes * 2
    ne = num_edges * 2
    n = nn + ne
    P = np.identity(n)
    
    q = np.zeros(n)
    q[:(nn)] = -2.0* theta

    T = np.identity(n)
    h = np.zeros(n)+1
    h[:nn] = theta 

    A = np.zeros([nn,n])
    edge_count = nn
    for i in range(num_nodes):
        A[i,i] = 1 
        A[num_nodes+i,num_nodes+i] = 1
        neighbor_size = len([k for k in G.neighbors(i)])
        for j in range(neighbor_size):
            A[i,(edge_count+j)] = -1
            A[num_nodes+i,(edge_count+j)] = -1
        edge_count = edge_count + neighbor_size
    
    b = np.repeat(0,nn)
    lb = np.repeat(0,n)
    x = solve_qp(P, q, T, h, A,b,lb)

    deg_list = []
    for i in range(num_nodes):
        deg_list.append(int(x[i].round()))
    deg_seq = sorted(deg_list,reverse=False)
    return deg_seq

# Raskhodnikova & Smith, Arxiv'15
# Slow because of quadratic programming; we only run it on the mini facebook set
# Advantage: automatically adapts to the graph structure
def nodedp_flowgraph_deg_seq_lap(G, max_deg, epsilon, beta): 
    # beta: an upper bound on the desired probability of a bad outcome
    true_deg_seq = util.get_sorted_deg_seq( G )
    num_nodes = len( G.nodes() )

    # Algorithm 2: generalized exponential mechanism for selecting threshold theta
    print("Running generalized exponential mechanism")
    theta_candidates = [2**i for i in range(int(np.log2(num_nodes))+1)]
    k = len( theta_candidates )
    t = 2 * np.log( k / beta ) / epsilon
    errors = []
    sens = []
    for theta in theta_candidates:
        sens.append( 3 * theta ) # an upper bound suffices
        degSeq = flowgraph( G, theta )
        error = sum( abs( np.subtract( degSeq, true_deg_seq ) ) ) + (6 * theta**2)/epsilon
        errors.append( error )

    prob_list = []
    for i in range( k ):
        s = []
        for j in range( k ):
            if not i == j:
                s.append( (errors[i]- errors[j] + t*sens[i] - t*sens[j] )/ (sens[i]+sens[j]))
        score = max( s )
        prob_list.append( np.exp( epsilon * score / 2.0 ))
    theta = theta_candidates[util.sample_prob_list(prob_list)]
    

    #Flowgraph algorithm to obtain a degree sequence with max degree is bounded by theta
    print("Selected optimal theta")
    deg_seq = flowgraph(G,theta)

    #Convert degSeq to degHis and add noise 
    sens = 4.0 * theta  
    deg_his = util.deg_seq_to_deg_his(deg_seq, theta)
    noisy_deg_his = util.add_laplace(deg_his, sens, epsilon)
    noisy_deg_his = util.extend_his(noisy_deg_his, max_deg)
    
    # postprocess (THE PAPER DOES NOT HAVE THIS STEP)
    noisy_deg_his = util.post_process_pdf(noisy_deg_his, num_nodes)
    return noisy_deg_his


### Simple Laplace mechanisms

In [10]:
#Naive laplace mechanism with postprocessing
def nodedp_deg_his_lap(G, max_deg, epsilon): #O(nodes)
    deg_his = util.get_deg_his(G, max_deg)
    sens = 2.0 * (max_deg + 1)
    noisy_histogram = util.add_laplace(deg_his, sens, epsilon)
#     noisy_histogram2 = util.post_process_pdf(noisy_histogram, len(G.nodes()))
    return noisy_histogram

# Adapting Hay et al. ICDM'09, Proserpio et al. WOSN'12 (wPINQ)
# Add laplace noise to degree sequence, post-process, convert to degree histogram
def nodedp_deg_seq_lap(G, max_deg, epsilon):
    deg_seq = np.array(util.get_sorted_deg_seq(G))
    sens = 1.0* (max_deg + 1)
    noisy_deg_seq = util.add_laplace(deg_seq, sens, epsilon)
    noisy_deg_seq = util.post_process_cdf(noisy_deg_seq, len(G.nodes())) # fit to isotonic regression
    noisy_histogram = util.deg_seq_to_deg_his(noisy_deg_seq, max_deg)
    return noisy_histogram


### Shiva et al. TCC'13: Node truncation

In [11]:
# Shiva et al. TCC'13
# Node truncation: remove nodes with degree > theta, add cauchy noise

from scipy.stats import cauchy

def nodedp_node_trun_smooth(G, max_deg, epsilon, theta): # O(num nodes)
    epsilon_deg = epsilon 
    
    #node truncation: remove nodes with degree > theta 
    G_copy = G.copy()
    nodes_to_truncate = [n for n,d in G_copy.degree() if d > theta]
    G_copy.remove_nodes_from(nodes_to_truncate)

    #smooth bound: (Prop 6.1, Algo 3)
    num_nodes = len(G.nodes())
    beta = epsilon / (np.sqrt(2.0) * (theta+1))   
    r = np.log(1.0*num_nodes/beta)
    l = len([n for n,d in G_copy.degree() if (d >= theta-r and d<=theta+r)])
    smooth_sens = l+1.0/beta + 1
    
    #add cauchy noise with epsilon_deg
    scale = 2*np.sqrt(2.0)*theta/epsilon_deg*smooth_sens
    true_his = util.get_deg_his(G_copy, theta)
    noisy_his = true_his + cauchy.rvs(0,scale=smooth_sens,size=len(true_his))
    noisy_his = util.extend_his(noisy_his, max_deg)
    
    #postprocess (TCC DOES NOT HAVE THIS STEP)
    noisy_his = util.post_process_pdf(noisy_his, num_nodes)
    
    return noisy_his


In [12]:
def baseline(G, max_deg, epsilon):
#     deg_his = util.get_deg_his(G, max_deg)
#     sens = 2.0 * (max_deg + 1)
#     noisy_histogram = util.add_laplace(deg_his, sens, epsilon)
#     noisy_histogram2 = util.post_process_pdf(noisy_histogram, len(G.nodes()))
#     noisy_hist = []
    for i in range(max_deg):
        noisy_hist.append(0)
    return noisy_hist

### Evaluation

### Node DP Degree Distribution Accuracy plots

In [ ]:
import constants
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

algos = [
            "nodedp_deg_his_lap",
             "nodedp_deg_seq_lap", 
             "nodedp_node_trun_smooth", 
             "nodedp_add_edge_deg_his_part_lap", 
             "nodedp_add_edge_deg_cum_lap",
             "nodedp_add_edge_deg_cum_lap_variant",
#              "nodedp_flowgraph_deg_seq_lap" # this is slow, we only ran it on the minifacebook dataset
]


epsilon_list = [0.01,0.02,0.05,0.1,0.2,0.5,1.0,2.0,5.0,10.0]
repeats = 10

data_dir = "data/" # REPLACE WITH YOUR DATASET DIRECTORY
data_key = 0
data_file = data_dir + constants.DATASETS[data_key]+'.txt'
data_file="frontend/Datasets/facebook_combined.txt"

print("data: ", data_file)
G = nx.read_edgelist(data_file, nodetype=int)

num_nodes = len(G.nodes()) # assume this is given
max_deg = num_nodes - 1  

true_histogram = util.get_deg_his(G, max_deg)

max_theta = min(num_nodes-1,200)
theta_candidates = [i+1 for i in range(max_theta)]

errors = []
stds = []
for algo in algos:
    print("algo: ", algo)
    algo_errors = []
    algo_stds = []
    for epsilon in epsilon_list:
        temp_errors = []
        for i in range(repeats):
            res = np.zeros(max_deg + 1)
            if algo == "nodedp_deg_his_lap":
                res = nodedp_deg_his_lap(G,max_deg,epsilon)
            elif algo == "nodedp_deg_seq_lap":
                res = nodedp_deg_seq_lap(G,max_deg,epsilon)
            elif algo == "nodedp_node_trun_smooth":
                res = nodedp_node_trun_smooth(G,max_deg,epsilon,max_theta)
            elif algo == "nodedp_add_edge_deg_his_part_lap":
                r_candidates = [1.025,1.05,1.1,1.2,1.4,1.8] #r determines the partition (g_i = {k:r^{i-1}<= k < r^i})
                (res,theta,r) = nodedp_add_edge_deg_his_part_lap(G, max_deg, epsilon, theta_candidates, r_candidates) 
            elif algo == "nodedp_add_edge_deg_cum_lap":
                (res,theta) = nodedp_add_edge_deg_cum_lap(G,max_deg,epsilon,theta_candidates)
            elif algo == "nodedp_add_edge_deg_cum_lap_variant":
                res = nodedp_add_edge_deg_cum_lap_variant(G,max_deg,epsilon,theta_candidates)
            elif algo == "nodedp_flowgraph_deg_seq_lap":
                beta = 0.3 # unspecified in the paper
                res = nodedp_flowgraph_deg_seq_lap(G,max_deg,epsilon, beta) #works for small graph
            else:
                print("no valid algo")

            noisy_pdf = res / num_nodes
        
            error = util.dif_deg_his_L1(true_histogram/num_nodes, noisy_pdf)
            print("error: ", error)
            temp_errors.append(error)
        algo_errors.append(np.mean(temp_errors))
        algo_stds.append(np.std(temp_errors))
        
    errors.append(algo_errors)
    stds.append(algo_stds)


In [ ]:
# plot errors and standard deviation

params = {'nodedp_add_edge_deg_cum_lap':['x','red'], 'nodedp_add_edge_deg_cum_lap_variant': ['o','green'],
          'nodedp_add_edge_deg_his_part_lap':['>','orange'], 'nodedp_node_trun_smooth':[',','blue'],
          'nodedp_deg_his_lap':['.','pink'], 'nodedp_deg_seq_lap':['.','yellow'], 
         }

fig = plt.figure()
for i in range(len(errors)):
    algo = algos[i]
    err = errors[i]
    st_dev = stds[i]
    plt.errorbar(x=epsilon_list, y=err, yerr=st_dev,label=algo,c=params[algo][1],marker=params[algo][0])
plt.legend(bbox_to_anchor=(1.1, 1))
plt.xlabel('Epsilon')
plt.ylabel('L1 Error')
plt.ylim([0,2.5])

